In [1]:
using Distributed
procs_to_use = 5
if nprocs() <= procs_to_use
    addprocs(procs_to_use-nprocs())
end
;

In [2]:
@everywhere using
    QuantumStates,
    OpticalBlochEquations,
    DifferentialEquations,
    UnitsToValue,
    StructArrays,
    StaticArrays,
    LoopVectorization,
    Parameters,
    MutableNamedTuples,
    StatsBase, 
    Plots, 
    Distributions
;

Precompiling QuantumStates
        Info Given QuantumStates was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
  ? QuantumStates
[ Info: Precompiling QuantumStates [17f0441f-15e0-42ae-a101-302633ff8f0f]
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
[ Info: Skipping precompilation since __precompile__(false). Importing QuantumStates [17f0441f-15e0-42ae-a101-302633ff8f0f].


      From worker 3:	WARNING: Method definition TDM(Any, Any) in module QuantumStates at C:\Google Drive\github\QuantumStates\src\HundsCaseB_LinearMolecule.jl:327 overwritten at C:\Google Drive\github\QuantumStates\src\HundsCaseB_LinearMolecule.jl:328.
      From worker 3:	ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
      From worker 5:	WARNING: Method definition TDM(Any, Any) in module QuantumStates at C:\Google Drive\github\QuantumStates\src\HundsCaseB_LinearMolecule.jl:327 overwritten at C:\Google Drive\github\QuantumStates\src\HundsCaseB_LinearMolecule.jl:328.
      From worker 5:	ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
      From worker 4:	WARNING: Method definition TDM(Any, Any) in module QuantumStates at C:\Google Drive\github\QuantumStates\src\HundsCaseB_LinearMolecule.jl:327 overwritten at C:\Google Dr

In [3]:
@everywhere include("helper_functions.jl")
;

# Define molecular states for the laser cooling transition

In [4]:
# Define constants for the laser cooling transition
@everywhere begin
    @consts begin
        λ = 626e-9             # wavelength of transition
        Γ = 2π * 6.4e6         # transition linewidth
        m = @with_unit 57 "u"  # mass of CaOH
        k = 2π / λ             # wavenumber of transition
    end
end
;

In [5]:
@everywhere include("CaOH_X(000).jl")
@everywhere include("CaOH_A(000).jl")
;

In [6]:
@everywhere begin
    H = CombinedHamiltonian([X_state_ham, A_state_ham])
    evaluate!(H)
    QuantumStates.solve!(H)
    update_basis_tdms!(H)
    update_tdms!(H)
    
    ground_state_idxs = 1:12
    excited_state_idxs = 17:20
    states_idxs = [ground_state_idxs; excited_state_idxs]
    
    ground_states = H.states[ground_state_idxs]
    excited_states = H.states[excited_state_idxs]
    
    d = H.tdms[states_idxs, states_idxs, :]
    states = H.states[states_idxs]
end

In [7]:
@everywhere begin
    Zeeman_x(state, state′) = (Zeeman(state, state′,-1) - Zeeman(state, state′,1))/sqrt(2)
    Zeeman_y(state, state′) = im*(Zeeman(state, state′,-1) + Zeeman(state, state′,1))/sqrt(2)
    Zeeman_z(state, state′) = Zeeman(state, state′, 0)
    
    Zeeman_x_mat = StructArray(operator_to_matrix_zero_padding2(Zeeman_x, ground_states, excited_states) .* (1e-4 * gS * μB * (2π/Γ) / h))
    Zeeman_y_mat = StructArray(operator_to_matrix_zero_padding2(Zeeman_y, ground_states, excited_states) .* (1e-4 * gS * μB * (2π/Γ) / h))
    Zeeman_z_mat = StructArray(operator_to_matrix_zero_padding2(Zeeman_z, ground_states, excited_states) .* (1e-4 * gS * μB * (2π/Γ) / h))
end
;

# Define experimental parameters

In [8]:
@everywhere begin
     
    Isat = π*h*c*Γ/(3λ^3)
    P = 0.5 * 0.7 * 13.1e-3 # 13.1 mW/1 V, factor of 0.5 to match scattering rates
    I = 2P / (π * 5e-3^2) # 8 mm 1/e^2 diameter beams
    s = I / Isat
    
    # Define B field gradient and ramp time
    B_grad_start = 0.
    B_grad_end = 78.
    ramp_time = 4e-3 / (1/Γ)
    detuning = +7.6
    δ1 = +0.0
    δ2 = +0.0
    δ3 = -1.00
    δ4 = +0.75
    
    Δ1 = 1e6 * (detuning + δ1)
    Δ2 = 1e6 * (detuning + δ2)
    Δ3 = 1e6 * (detuning + δ3)
    Δ4 = 1e6 * (detuning + δ4)
    
    s1 = 0.00I/Isat
    s2 = 0.37I/Isat
    s3 = 0.28I/Isat
    s4 = 0.35I/Isat
    
    T_initial = 35e-6
    
    params = MutableNamedTuple(
        pol1_x=σ⁺, pol2_x=σ⁻, pol3_x=σ⁺, pol4_x=σ⁻,
        s1=s1, s2=s2, s3=s3, s4=s4,
        Δ1=Δ1, Δ2=Δ2, Δ3=Δ3, Δ4=Δ4,
        
        B_grad_start = B_grad_start,
        B_grad_end = B_grad_end,
        B_ramp_time = ramp_time,
        
        s_ramp_time = ramp_time,
        s_ramp_factor = 1.,

        photon_budget = rand(Geometric(1/13500)),
        
        x_dist = Normal(0,700e-6),
        y_dist = Normal(0,700e-6),
        z_dist = Normal(0,400e-6),

        vx_dist = Normal(0, sqrt(kB*T_initial/2m)),
        vy_dist = Normal(0, sqrt(kB*T_initial/2m)),
        vz_dist = Normal(0, sqrt(kB*T_initial/2m)),

        Zeeman_Hx = Zeeman_x_mat,
        Zeeman_Hy = Zeeman_y_mat,
        Zeeman_Hz = Zeeman_z_mat
    )
end

In [9]:
@everywhere begin
    include("define_lasers.jl")
    lasers = define_lasers(
        states,
        params.s1,
        params.s2,
        params.s3,
        params.s4,
        params.Δ1,
        params.Δ2,
        params.Δ3,
        params.Δ4,
        params.pol1_x,
        params.pol2_x,
        params.pol3_x,
        params.pol4_x,
        params.s_ramp_time,
        params.s_ramp_factor
    )
end
;

In [ ]:
@everywhere function update_H_and_∇H(H, p, r, t)

    Zeeman_Hz = p.params.Zeeman_Hz
    Zeeman_Hx = p.params.Zeeman_Hx
    Zeeman_Hy = p.params.Zeeman_Hy

    # set ramp factor
    scalar = t/p.params.B_ramp_time
    scalar = min(scalar, 1.0)
    
    B_grad = scalar * (p.params.B_grad_end - p.params.B_grad_start) + p.params.B_grad_start
    
    gradient_x = +scalar * B_grad * 1e2 / k / 2
    gradient_y = +scalar * B_grad * 1e2 / k / 2
    gradient_z = -scalar * B_grad * 1e2 / k
    
    Bx = gradient_x * r[1]
    By = gradient_y * r[2]
    Bz = gradient_z * r[3]
    
    @turbo for i in eachindex(H)
        H.re[i] = Bx * Zeeman_Hx.re[i] + By * Zeeman_Hy.re[i] + Bz * Zeeman_Hz.re[i]
        H.im[i] = Bx * Zeeman_Hx.im[i] + By * Zeeman_Hy.im[i] + Bz * Zeeman_Hz.im[i]
    end

    ∇H = SVector{3, Float64}(0,0,0)
  
    return ∇H
end

## Simulate trajectory for single particle

In [ ]:
@everywhere begin
    t_start = 0.0
    t_end   = 6e-3
    t_span  = (t_start, t_end) ./ (1/Γ)

    ψ₀ = zeros(ComplexF64, length(states))
    ψ₀[1] = 1.0

    diffusion_constant_simple = ones(3) .* (ħ^2*k^2/3) ./ (m*ħ*Γ)
    
    particle = Particle()
    particle.r = (0,0,0.3e-3) ./ (1/k)
    
    p = schrodinger_stochastic(particle, states, lasers, d, ψ₀, m/(ħ*k^2/Γ), 4; params=params, λ=λ, Γ=Γ, update_H_and_∇H=update_H_and_∇H, diffusion_constant=diffusion_constant_simple)
    cb1 = ContinuousCallback(condition, stochastic_collapse!, nothing, save_positions=(false,false))
    cb2 = DiscreteCallback(terminate_condition, terminate!)
    cbs = CallbackSet(cb1,cb2)
    prob = ODEProblem(ψ_stochastic_diffusion!, p.ψ, t_span, p; alg=DP5(), maxiters=200000000, callback=cbs, reltol=1e-3, saveat=1000)
end
@time sol = DifferentialEquations.solve(prob)
;

 22.419810 seconds (30.02 M allocations: 1.928 GiB, 6.94% gc time, 62.67% compilation time)


In [21]:
ensemble_prob = EnsembleProblem(prob; prob_func=prob_func)
@time ensemble_sol = solve(ensemble_prob, EnsembleDistributed(); trajectories=10)
;

 73.944810 seconds (448.67 k allocations: 43.531 MiB, 0.01% compilation time: 100% of which was recompilation)


In [18]:
using DiffEqGPU, CUDA

In [23]:
ensemble_prob = EnsembleProblem(prob; prob_func=prob_func)
@time ensemble_sol = solve(ensemble_prob, Tsit5(), EnsembleGPUArray(CUDA.CUDABackend()); trajectories=10)
;

LoadError: InvalidIRError: compiling MethodInstance for DiffEqGPU.gpu_gpu_kernel(::KernelAbstractions.CompilerMetadata{KernelAbstractions.NDIteration.DynamicSize, KernelAbstractions.NDIteration.DynamicCheck, Nothing, CartesianIndices{1, Tuple{Base.OneTo{Int64}}}, KernelAbstractions.NDIteration.NDRange{1, KernelAbstractions.NDIteration.DynamicSize, KernelAbstractions.NDIteration.DynamicSize, CartesianIndices{1, Tuple{Base.OneTo{Int64}}}, CartesianIndices{1, Tuple{Base.OneTo{Int64}}}}}, ::typeof(ψ_stochastic_diffusion!), ::CuDeviceMatrix{ComplexF64, 1}, ::CuDeviceMatrix{ComplexF64, 1}, ::CuDeviceMatrix{MutableNamedTuple{(:H, :H₀, :∇H, :ψ, :dψ, :ψ_soa, :dψ_soa, :ω, :eiωt, :Js, :states, :fields, :r0, :r, :v, :d, :d_nnz, :λ, :k, :Γ, :E, :E_k, :ds, :ds_state1, :ds_state2, :params, :mass, :update_H_and_∇H, :populations, :n_scatters, :save_counter, :n_states, :n_ground, :n_excited, :trajectory, :decay_dist, :time_to_decay, :last_decay_time, :∇H_x, :∇H_y, :∇H_z, :∇H_x_ψ, :∇H_y_ψ, :∇H_z_ψ, :H_nh